In [35]:
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
import numpy as np
import PIL
from PIL import Image
import keras

In [36]:
#3层卷积
imgSize = 1024  #图片是512*512
imgChan = 3    #3通道
kernelInit = 'uniform'  #卷积核初始化方式
nHiddenUnits = 50
maxpoolSize = (4,4)
nFilters = 100 #130 before 
kernelSize = (4,4)
inputStrides = 1
#costFunction = 'categorical_crossentropy' #代价函数:不同于损失函数，代价函数在整个训练集上计算
early_stop_delta = 0.01 # 0.01 change or above is considered improvement
early_stop_patience = 10 

model=Sequential()

model.add(Conv2D(filters=nFilters, kernel_size = kernelSize ,strides= inputStrides,
                 input_shape=(imgSize,imgSize,3),kernel_initializer= kernelInit))
model.add(Activation('relu'))



model.add(MaxPooling2D(pool_size=maxpoolSize))


model.add(Conv2D(filters=nFilters, kernel_size = kernelSize ,strides= inputStrides,
                 kernel_initializer= kernelInit))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=maxpoolSize))


model.add(Conv2D(filters=nFilters, kernel_size = kernelSize ,strides= inputStrides,
                 kernel_initializer= kernelInit))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=maxpoolSize))

model.add(Flatten())
model.add(Dense(nHiddenUnits))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid')) #二分类

earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=early_stop_delta, patience=early_stop_patience, verbose=2, mode='auto')

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 1021, 1021, 100)   4900      
_________________________________________________________________
activation_21 (Activation)   (None, 1021, 1021, 100)   0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 255, 255, 100)     0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 252, 252, 100)     160100    
_________________________________________________________________
activation_22 (Activation)   (None, 252, 252, 100)     0         
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 63, 63, 100)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 60, 60, 100)      

In [37]:
#定义 train_generator:
def train_generator():
    while 1:
        with open("../../data/ImgProcessed/training/m/names.txt") as file:
            for line in file.readlines():
                img = Image.open("../../data/ImgProcessed/training/m/"+line.strip("\n"))
                img = img.resize((imgSize,imgSize))
                #print(model.predict(np.array(img).reshape((1,imgSize,imgSize,3))))
                yield (np.array(img).reshape((1,imgSize,imgSize,3))/255,np.ones((1,1)))
                #1是煤
                #imgs.append(img)
        with open("../../data/ImgProcessed/training/s/names.txt") as file:
            for line in file.readlines():
                img = Image.open("../../data/ImgProcessed/training/s/"+line.strip("\n"))
                img = img.resize((imgSize,imgSize))
                #print(model.predict(np.array(img).reshape((1,imgSize,imgSize,3))))
                yield (np.array(img).reshape((1,imgSize,imgSize,3))/255,np.zeros((1,1)))
                #0是煤矸石
                #imgs.append(img)
def validation_generator():
    while 1:
        with open("../../data/ImgProcessed/validation/m/names.txt") as file:
            for line in file.readlines():
                img = Image.open("../../data/ImgProcessed/validation/m/"+line.strip("\n"))
                img = img.resize((imgSize,imgSize))
                yield (np.array(img).reshape((1,imgSize,imgSize,3))/255,np.ones((1,1)))
                #1是煤
        with open("../../data/ImgProcessed/validation/s/names.txt") as file:
            for line in file.readlines():
                img = Image.open("../../data/ImgProcessed/validation/s/"+line.strip("\n"))
                img = img.resize((imgSize,imgSize))
                yield (np.array(img).reshape((1,imgSize,imgSize,3))/255,np.zeros((1,1)))
                #0是煤矸石

In [38]:
test = train_generator()
x,y = test.__next__()
print(x)

[[[[0.8        0.83529412 0.81568627]
   [0.80392157 0.83921569 0.81960784]
   [0.80784314 0.84313725 0.82352941]
   ...
   [0.7254902  0.75294118 0.72156863]
   [0.7254902  0.75294118 0.72156863]
   [0.72941176 0.75686275 0.7254902 ]]

  [[0.8        0.83529412 0.81568627]
   [0.80392157 0.83921569 0.81960784]
   [0.80392157 0.83921569 0.81960784]
   ...
   [0.7254902  0.75294118 0.72156863]
   [0.7254902  0.75294118 0.72156863]
   [0.72941176 0.75686275 0.7254902 ]]

  [[0.8        0.83529412 0.81568627]
   [0.80392157 0.83921569 0.81960784]
   [0.79607843 0.83137255 0.81176471]
   ...
   [0.7254902  0.75294118 0.72156863]
   [0.7254902  0.75294118 0.72156863]
   [0.72941176 0.75686275 0.7254902 ]]

  ...

  [[0.74901961 0.77647059 0.74901961]
   [0.74901961 0.77647059 0.74901961]
   [0.74509804 0.77254902 0.74509804]
   ...
   [0.71764706 0.74509804 0.71764706]
   [0.71764706 0.74509804 0.71764706]
   [0.72156863 0.74901961 0.72156863]]

  [[0.74901961 0.77647059 0.74901961]
   [0.7

In [42]:
model.compile(optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08),loss="binary_crossentropy",metrics=['binary_accuracy'])
history = model.fit_generator(train_generator(),steps_per_epoch=25,epochs=5,validation_data=validation_generator(),validation_steps=107)

Epoch 1/5
25/25 [==============================] - 158s 6s/step - loss: 1.1921e-07 - binary_accuracy: 1.0000 - val_loss: 7.1517 - val_binary_accuracy: 0.5514
Epoch 2/5
25/25 [==============================] - 158s 6s/step - loss: 1.1921e-07 - binary_accuracy: 1.0000 - val_loss: 7.1517 - val_binary_accuracy: 0.5514
Epoch 3/5
25/25 [==============================] - 156s 6s/step - loss: 1.1921e-07 - binary_accuracy: 1.0000 - val_loss: 8.4927 - val_binary_accuracy: 0.4673
Epoch 4/5
25/25 [==============================] - 159s 6s/step - loss: 1.1921e-07 - binary_accuracy: 1.0000 - val_loss: 8.7907 - val_binary_accuracy: 0.4486
Epoch 5/5
25/25 [==============================] - 149s 6s/step - loss: 0.6377 - binary_accuracy: 0.9600 - val_loss: 7.7477 - val_binary_accuracy: 0.5140
